In [1]:
import pandas as pd
import numpy as np
import re as re
import requests
from bs4 import BeautifulSoup

In [2]:
import chardet

lfile = './input/top250-00-19.csv'
def openDf(x):
    with open(x, 'rb') as f:
        result = chardet.detect(f.read())
    return pd.read_csv(x, encoding=result['encoding'])

df = openDf(lfile)
print(df.shape)

(4700, 10)


In [3]:
df.dtypes

Name             object
Position         object
Age               int64
Team_from        object
League_from      object
Team_to          object
League_to        object
Season           object
Market_value    float64
Transfer_fee      int64
dtype: object

In [4]:
df.head()
#df.groupby(['Transfer_fee', 'Season']).max().sort_index(ascending=False)

,Name,Position,Age,Team_from,League_from,Team_to,League_to,Season,Market_value,Transfer_fee
0,Luís Figo,Right Winger,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,2000-2001,NaN,60000000
1,Hernán Crespo,Centre-Forward,25,Parma,Serie A,Lazio,Serie A,2000-2001,NaN,56810000
2,Marc Overmars,Left Winger,27,Arsenal,Premier League,FC Barcelona,LaLiga,2000-2001,NaN,40000000
3,Gabriel Batistuta,Centre-Forward,31,Fiorentina,Serie A,AS Roma,Serie A,2000-2001,NaN,36150000
4,Nicolas Anelka,Centre-Forward,21,Real Madrid,LaLiga,Paris SG,Ligue 1,2000-2001,NaN,34500000


In [5]:
null_cols = df.isnull().sum()
null_cols

Name               0
Position           0
Age                0
Team_from          0
League_from        0
Team_to            0
League_to          0
Season             0
Market_value    1260
Transfer_fee       0
dtype: int64

In [6]:
def nullOut(x):
    null_cols = x.isnull().sum()
    return x.drop((list(null_cols[null_cols>1000].index)), axis=1)
df=nullOut(df)

In [7]:
def dropCol(x, y):
    return x.drop(columns=y)

df=dropCol(df,['Position','League_to', 'Age'])
    

In [8]:
def convMill(x, y):
    x[y]=x[y]/10**6
    return x

df = convMill(df, 'Transfer_fee')
df

,Name,Team_from,League_from,Team_to,Season,Transfer_fee
0,Luís Figo,FC Barcelona,LaLiga,Real Madrid,2000-2001,60.00
1,Hernán Crespo,Parma,Serie A,Lazio,2000-2001,56.81
2,Marc Overmars,Arsenal,Premier League,FC Barcelona,2000-2001,40.00
3,Gabriel Batistuta,Fiorentina,Serie A,AS Roma,2000-2001,36.15
4,Nicolas Anelka,Real Madrid,LaLiga,Paris SG,2000-2001,34.50
...,...,...,...,...,...,...
4695,Jasmin Kurtic,Atalanta,Serie A,SPAL,2018-2019,4.80
4696,Tchê Tchê,Palmeiras,Série A,Dynamo Kyiv,2018-2019,4.80
4697,Silvan Widmer,Udinese Calcio,Serie A,FC Basel,2018-2019,4.50
4698,Yuya Osako,1. FC Köln,2.Bundesliga,Werder Bremen,2018-2019,4.50


In [9]:
df.head()

,Name,Team_from,League_from,Team_to,Season,Transfer_fee
0,Luís Figo,FC Barcelona,LaLiga,Real Madrid,2000-2001,60.00
1,Hernán Crespo,Parma,Serie A,Lazio,2000-2001,56.81
2,Marc Overmars,Arsenal,Premier League,FC Barcelona,2000-2001,40.00
3,Gabriel Batistuta,Fiorentina,Serie A,AS Roma,2000-2001,36.15
4,Nicolas Anelka,Real Madrid,LaLiga,Paris SG,2000-2001,34.50


In [10]:
print(sorted(set(df['League_from'])))


[' Argentina', ' Australia', ' Brazil', ' Bulgaria', ' Canada', ' Chile', ' China', ' Colombia', ' Croatia', ' Czech Republic', ' Denmark', ' Ecuador', ' England', ' Finland', ' France', ' Ghana', ' Iran', ' Korea, South', ' Latvia', ' Mexico', ' Moldova', ' Paraguay', ' Peru', ' Portugal', ' Qatar', ' Romania', ' Russia', ' Saudi Arabia', ' Scotland', ' Serbia', ' Slovakia', ' South Africa', ' Spain', ' Sweden', ' Tunisia', ' Ukraine', ' United Arab Emirates', ' United States', ' Uruguay', ' Venezuela', '1.Bundesliga', '1.Division', '1.HNL', '1.Lig', '1.Liga gr. 1', '2.Bundesliga', '2ª B - Grupo I', '2ª B - Grupo III', '3.Liga', 'A Grupa - Championship gr.', 'Allsvenskan', 'Auf-/Abstiegsrunde NLA/NLB', 'Botola Pro', 'Bundesliga', 'Challenge League', 'Championnat National', 'Championship', 'Ekstraklasa', 'Eliteserien', 'Eredivisie', 'First Division', 'HET Liga', 'J1 - 2nd Stage', 'J1 League', 'J2 League', 'Jupiler Pro League', 'K League 1', 'Korean FA Cup', 'LaLiga', 'LaLiga2', 'League

In [11]:
probando = df[(df['League_from'] == 'Bundesliga')]
probando

,Name,Team_from,League_from,Team_to,Season,Transfer_fee
484,Andrés Fleurquín,Sturm Graz,Bundesliga,Galatasaray,2001-2002,2.0
1343,Andreas Ivanschitz,Rapid Vienna,Bundesliga,RB Salzburg,2005-2006,4.0
1454,Rabiu Afolabi,Austria Vienna,Bundesliga,FC Sochaux,2005-2006,2.0
2377,Erwin Hoffer,Rapid Vienna,Bundesliga,SSC Napoli,2009-2010,5.0
2568,Marc Janko,RB Salzburg,Bundesliga,FC Twente,2010-2011,7.0
2625,Nikica Jelavic,Rapid Vienna,Bundesliga,Rangers,2010-2011,4.9
2922,Nacer Barazite,Austria Vienna,Bundesliga,Monaco,2011-2012,4.5
3489,Sadio Mané,RB Salzburg,Bundesliga,Southampton,2014-2015,23.0
3532,Kevin Kampl,RB Salzburg,Bundesliga,Bor. Dortmund,2014-2015,12.0
3539,Alan,RB Salzburg,Bundesliga,GZ Evergrande,2014-2015,11.1


In [12]:
def generaFiltros(x, y, ligas):
    #filtrosLigas = dict()
    #for liga in ligas:
        #filtrosLigas[liga] = x.loc[(x['League_from'] == liga)]
    #for liga in ligas:
    prob = x[(x[y].isin(ligas))]
    return prob

z = ['Bundesliga','LaLiga','Premier League','League One', 'Serie A', 'Eredivisie']
df = generaFiltros(df, 'League_from', z)

In [13]:
df

,Name,Team_from,League_from,Team_to,Season,Transfer_fee
0,Luís Figo,FC Barcelona,LaLiga,Real Madrid,2000-2001,60.00
1,Hernán Crespo,Parma,Serie A,Lazio,2000-2001,56.81
2,Marc Overmars,Arsenal,Premier League,FC Barcelona,2000-2001,40.00
3,Gabriel Batistuta,Fiorentina,Serie A,AS Roma,2000-2001,36.15
4,Nicolas Anelka,Real Madrid,LaLiga,Paris SG,2000-2001,34.50
...,...,...,...,...,...,...
4689,Jens Odgaard,Inter,Serie A,Sassuolo,2018-2019,5.00
4690,Marco Carraro,Inter,Serie A,Atalanta,2018-2019,5.00
4692,Giangiacomo Magnani,Juventus,Serie A,Sassuolo,2018-2019,5.00
4695,Jasmin Kurtic,Atalanta,Serie A,SPAL,2018-2019,4.80


In [14]:
print(sorted(list(set(df['Season']))))

['2000-2001', '2001-2002', '2002-2003', '2003-2004', '2004-2005', '2005-2006', '2006-2007', '2007-2008', '2008-2009', '2009-2010', '2010-2011', '2011-2012', '2012-2013', '2013-2014', '2014-2015', '2015-2016', '2016-2017', '2017-2018', '2018-2019']


In [15]:
'''
def convertSeason(DF, column, length):
    se = sorted(list(set(DF[column])))
    lista=[]
    for s in se:
        lista.append(int(s[:length]))
    return sorted(lista)

lista = convertSeason(df, 'Season', 4)
print(lista)
'''

"\ndef convertSeason(DF, column, length):\n    se = sorted(list(set(DF[column])))\n    lista=[]\n    for s in se:\n        lista.append(int(s[:length]))\n    return sorted(lista)\n\nlista = convertSeason(df, 'Season', 4)\nprint(lista)\n"

In [16]:
#for col in df['Season']:
    #print(col)

In [17]:

def convertSeason(DF, column, length):
    se = sorted(list(set(DF[column])))
    lista=[]
    for s in se:
        lista.append(s[:length])
    for col in DF[column]:
        for i in range(0,len(lista)):
            if col == se[i]:
                DF[column] = DF[column].replace({col: lista[i]})
    return DF
    

df = convertSeason(df, 'Season', 4)

/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [18]:
def changeType(DF, column, types):
    DF[column] = DF[column].astype(types)
    return DF

df = changeType(df, 'Season', int)


/usr/local/Cellar/ipython/7.8.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
df

,Name,Team_from,League_from,Team_to,Season,Transfer_fee
0,Luís Figo,FC Barcelona,LaLiga,Real Madrid,2000,60.00
1,Hernán Crespo,Parma,Serie A,Lazio,2000,56.81
2,Marc Overmars,Arsenal,Premier League,FC Barcelona,2000,40.00
3,Gabriel Batistuta,Fiorentina,Serie A,AS Roma,2000,36.15
4,Nicolas Anelka,Real Madrid,LaLiga,Paris SG,2000,34.50
...,...,...,...,...,...,...
4689,Jens Odgaard,Inter,Serie A,Sassuolo,2018,5.00
4690,Marco Carraro,Inter,Serie A,Atalanta,2018,5.00
4692,Giangiacomo Magnani,Juventus,Serie A,Sassuolo,2018,5.00
4695,Jasmin Kurtic,Atalanta,Serie A,SPAL,2018,4.80


In [20]:
def changeNamecol(DF, column, newname):
    DF = DF.rename(columns={column: newname})
    return DF

df = changeNamecol(df, 'Season', 'Summer')

In [21]:
df["Summer"].value_counts()

2014    119
2011    118
2008    115
2009    114
2017    114
2016    110
2012    107
2015    104
2007    101
2013     99
2000     98
2018     96
2010     95
2001     90
2005     88
2006     83
2004     79
2002     64
2003     62
Name: Summer, dtype: int64

In [22]:
df.columns

Index(['Name', 'Team_from', 'League_from', 'Team_to', 'Summer',
       'Transfer_fee'],
      dtype='object')

In [23]:
df = changeType(df, 'Summer', str)


In [24]:
df['Name-season']=df['Name']+df['Summer']
df

,Name,Team_from,League_from,Team_to,Summer,Transfer_fee,Name-season
0,Luís Figo,FC Barcelona,LaLiga,Real Madrid,2000,60.00,Luís Figo2000
1,Hernán Crespo,Parma,Serie A,Lazio,2000,56.81,Hernán Crespo2000
2,Marc Overmars,Arsenal,Premier League,FC Barcelona,2000,40.00,Marc Overmars2000
3,Gabriel Batistuta,Fiorentina,Serie A,AS Roma,2000,36.15,Gabriel Batistuta2000
4,Nicolas Anelka,Real Madrid,LaLiga,Paris SG,2000,34.50,Nicolas Anelka2000
...,...,...,...,...,...,...,...
4689,Jens Odgaard,Inter,Serie A,Sassuolo,2018,5.00,Jens Odgaard2018
4690,Marco Carraro,Inter,Serie A,Atalanta,2018,5.00,Marco Carraro2018
4692,Giangiacomo Magnani,Juventus,Serie A,Sassuolo,2018,5.00,Giangiacomo Magnani2018
4695,Jasmin Kurtic,Atalanta,Serie A,SPAL,2018,4.80,Jasmin Kurtic2018


In [25]:
df.to_csv('./output/TransferLigas.csv', header=True, index=False)

In [26]:
import requests

def getPage(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup

url = "https://es.wikipedia.org/wiki/Trofeo_Pichichi"
soup = getPage(url)
#soup


<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>Trofeo Pichichi - Wikipedia, la enciclopedia libre</title>
<script>document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Trofeo_Pichichi","wgTitle":"Trofeo Pichichi","wgCurRevisionId":119743022,"wgRevisionId":119743022,"wgArticleId":105464,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia:Artículos con datos por trasladar a Wikidata","Premios de fútbol de España","Premios individuales de fútbol en España","Premios otorgados por el Diario Marca","Anexos:Máximos goleadores por temporada"],"wgBreakFrames":!1,"wgPageContentLanguage":"es","wgPageContentModel":"wikitext","wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","enero","febrero","marzo","abril","mayo

In [27]:
'''
res = requests.get("https://es.wikipedia.org/wiki/Trofeo_Pichichi")
html = res.text
html[:20]
'''

'\nres = requests.get("https://es.wikipedia.org/wiki/Trofeo_Pichichi")\nhtml = res.text\nhtml[:20]\n'

In [29]:
'''
from bs4 import BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')
soup
'''

"\nfrom bs4 import BeautifulSoup\nsoup = BeautifulSoup(html, 'html.parser')\nsoup\n"

In [30]:
jugador = soup.select('#mw-content-text > div > center > table > tbody > tr:nth-child(77) > td:nth-child(2) > a')

In [31]:
goles = soup.select('#mw-content-text > div > center > table > tbody > tr:nth-child(77) > td:nth-child(6)')

In [32]:
temporada = soup.select('#mw-content-text > div > center > table > tbody > tr:nth-child(79) > td:nth-child(1) > a')


In [33]:
def getPichichi(soup, postable, row, cellSeason, cellPlayers, cellGoals):
    table = soup.find_all('table')[postable]
    rows = table.find_all('tr')[row:]
    pichichi = []
    for row in rows:
        cells = row.find_all('td')
        pichichi.append({
            "season": re.sub("[()\\n]", "", cells[cellSeason].find('a').text),
            "players": re.sub("[0-9]?[()\\n]", "", cells[cellPlayers].text),
            "goals": int(re.sub("[()\\n]", "", cells[cellGoals].text)[:2])       
        })
    return pd.DataFrame(pichichi)

coso = getPichichi(soup, 1, 76, 0, 1, 5)
coso

,season,players,goals
0,1999–00,Salva Ballesta,27
1,2000–01,Raúl González,24
2,2001–02,Diego Tristán,21
3,2002–03,Roy Makaay,29
4,2003–04,Ronaldo,24
5,2004–05,Diego Forlán [27]​,25
6,2005–06,Samuel Eto'o,26
7,2006–07,Ruud van Nistelrooy,25
8,2007–08,Dani Güiza,27
9,2008–09,Diego Forlán,32


In [34]:
def subThings(DF, column, text):
    DF[column] = DF[column].replace('\[?[0-9]+\]?', text)
    return DF

coso = subThings(coso, 'players', "")
coso

,season,players,goals
0,1999–00,Salva Ballesta,27
1,2000–01,Raúl González,24
2,2001–02,Diego Tristán,21
3,2002–03,Roy Makaay,29
4,2003–04,Ronaldo,24
5,2004–05,Diego Forlán [27]​,25
6,2005–06,Samuel Eto'o,26
7,2006–07,Ruud van Nistelrooy,25
8,2007–08,Dani Güiza,27
9,2008–09,Diego Forlán,32


In [35]:
print(list(range(2000,2019)))

[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]


In [36]:
def convSeas(DF, column, range1, range2):
    lista = list(range(range1,range2))
    DF[column]=lista
    return DF

campeones=convSeas(coso, 'season', 2000, 2020)

In [37]:
'''
tabla = soup.find_all('table')[1]
filas = tabla.find_all('tr')[77:]
filas
'''

"\ntabla = soup.find_all('table')[1]\nfilas = tabla.find_all('tr')[77:]\nfilas\n"

In [38]:
'''
#TABLA
goleadores = []
for fila in filas:
    celdas = fila.find_all('td')
    goleadores.append({
        "temporadas": re.sub("[()\\n]", "", celdas[0].find('a').text),
        "jugadores": re.sub("[0-9]?[()\\n]", "", celdas[1].text),
        "goles": int(re.sub("[()\\n]", "", celdas[5].text)[:2])       
    })
    
print(goleadores)
'''

'\n#TABLA\ngoleadores = []\nfor fila in filas:\n    celdas = fila.find_all(\'td\')\n    goleadores.append({\n        "temporadas": re.sub("[()\\n]", "", celdas[0].find(\'a\').text),\n        "jugadores": re.sub("[0-9]?[()\\n]", "", celdas[1].text),\n        "goles": int(re.sub("[()\\n]", "", celdas[5].text)[:2])       \n    })\n    \nprint(goleadores)\n'

In [39]:
#pd.DataFrame(goleadores)

In [40]:
cosita=getPage("http://www.lalanternadelpopolo.it/Calcio%20-%20Capocannonieri%20Campionato%20Italia%20Albo%20d'Oro%20Vincitori.htm")


In [41]:

def getCapo(soup):
    table = cosita.find_all('table')[3]
    rows = table.select('tr')[77:97]
    capo = []
    for row in rows:
        cells = row.find_all('td')
        capo.append({
            "season": cells[0].text.strip(),
            "players": cells[1].text.strip(),
            "goals": cells[2].text.strip()})
    return pd.DataFrame(capo)

capoDf = getCapo(cosita)
capoDf
                                                        






,season,players,goals
0,2000,Andriy Shevchenko,24
1,2001,Hernan Crespo,26
2,2002,David Trezeguet,24
3,2003,Christian Vieri,24
4,2004,Andriy Shevchenko,24
5,2005,Cristiano Lucarelli,24
6,2006,Luca Toni,31
7,2007,Francesco Totti,26
8,2008,Alessandro Del Piero,21
9,2009,Zlatan Ibrahimovic,25


In [42]:
#dos = capoDf[16].shift(1, axis = 1) 
#dos
#capoDf.iloc[16]
#capoDf.loc[16], ['season', 'players', 'goals']

In [43]:
def deleteRow(DF, row):
    DF = DF.drop(row)
    return DF

italianos = deleteRow(capoDf, 16)
#italianos

In [44]:
#def subThings(DF, column, text):
#    DF[column] = DF[column].replace('[A-Z]+?[a-z]+?\s?[A-Z]+?[a-z]+', text)
#    return DF

In [45]:
top = getPage('https://www.worldfootball.net/top_scorer/eng-premier-league/')


In [46]:
def getTop(soup):
    table = top.find_all('table')[0]
    rows = table.select('tr')[1:24]
    rows
    tp = []
    for row in rows:
        cells = row.find_all('td')
        tp.append({
            "season": cells[0].text.strip(),
            "players": cells[2].text.strip(),
            "goals": cells[5].text.strip()})
    return pd.DataFrame(tp).sort_values('season')

topPL = getTop(top)
topPL

,season,players,goals
11,,Carlos TÃ©vez,21
1,,Pierre-Emerick Aubameyang,22
2,,Mohamed Salah,22
22,1999/2000,Kevin Phillips,30
21,2000/2001,Jimmy Floyd Hasselbaink,23
20,2001/2002,Thierry Henry,24
19,2002/2003,Ruud van Nistelrooy,25
18,2003/2004,Thierry Henry,30
17,2004/2005,Thierry Henry,25
16,2005/2006,Thierry Henry,27


In [47]:
nope = deleteRow(topPL, [1,2,11])
#for season in nope['season']:
    #print(season)
  #  count=1999
nope['season']=nope['season'].replace('.+', "hola")
nope

,season,players,goals
22,1999/2000,Kevin Phillips,30
21,2000/2001,Jimmy Floyd Hasselbaink,23
20,2001/2002,Thierry Henry,24
19,2002/2003,Ruud van Nistelrooy,25
18,2003/2004,Thierry Henry,30
17,2004/2005,Thierry Henry,25
16,2005/2006,Thierry Henry,27
15,2006/2007,Didier Drogba,20
14,2007/2008,Cristiano Ronaldo,31
13,2008/2009,Nicolas Anelka,19


In [48]:
ingleses = convSeas(nope, 'season', 2000, 2020)

In [49]:
french = getPage('https://es.wikipedia.org/wiki/Anexo:M%C3%A1ximos_goleadores_de_la_Liga_Francesa')

def getL1(soup):
    table = soup.find_all('table')[0]
    rows = table.find_all('tr')[62:]
    fr = []
    for row in rows:
        cells = row.find_all('td')
        fr.append({
            "season": cells[0].text.strip(),
                "players": cells[1].text.strip(),
                "goals": cells[3].text.strip()})
    return pd.DataFrame(fr)

#getL1(french)

In [50]:
holl = getPage('https://es.wikipedia.org/wiki/Anexo:M%C3%A1ximos_goleadores_de_la_Eredivisie')
def getL1(soup):
    table = soup.find_all('table')[0]
    rows = table.find_all('tr')[44:]
    fr = []
    for row in rows:
        cells = row.find_all('td')
        fr.append({
            "season": cells[0].text.strip(),
                "players": cells[1].text.strip(),
                "goals": cells[2].text.strip()})
    return pd.DataFrame(fr)

#getL1(holl)

In [51]:
def getWiki(soup, postable, posrows, posSeason, posPlayers, posGoals):
    table = soup.find_all('table')[postable]
    rows = table.find_all('tr')[posrows:]
    wiki = []
    for row in rows:
        cells = row.find_all('td')
        wiki.append({
            "season": cells[posSeason].text.strip(),
                "players": cells[posPlayers].text.strip(),
                "goals": cells[posGoals].text.strip()})
    return pd.DataFrame(wiki)

holland = getWiki(holl, 0, 44, 0, 1, 2)

In [52]:
frenchs = getWiki(french, 0, 62, 0, 1, 3)

In [58]:
bund = getPage('https://es.wikipedia.org/wiki/Anexo:Estad%C3%ADsticas_de_la_Bundesliga_de_Alemania')
bundSc = getWiki(bund, 3, 37, 0, 1, 3)
#bundSc

In [59]:
holi=convSeas(holland, 'season', 2000, 2019)
holandeses = subChar(holi, 'players', 'ž', 'z')
#holandeses

In [60]:
fr = convSeas(frenchs, 'season', 2000, 2018)
#fr

In [61]:
alemanes = convSeas(bundSc, 'season', 2000, 2019)
alemanes.dtypes


season      int64
players    object
goals      object
dtype: object

In [62]:
def subChar(DF, column, old, new):
    DF[column] = DF[column].str.replace(old, new)
    return DF

alemanes = subChar(alemanes, 'players', 'ž', 'z')
alemanes = subChar(alemanes, 'players', ' Martin Max', '')
alemanes = subChar(alemanes, 'players', ' Ebbe Sand', '')
alemanes = subChar(alemanes, 'players', ' Thomas Christiansen', '')
#alemanes


In [63]:
fr=subChar(fr, 'players', 'CisséPauleta', 'Cissé')
fr=subChar(fr, 'players', 'ć', 'c')
franceses=subChar(fr, 'players', ' Nené', '')
#franceses

In [64]:
ingleses=subChar(ingleses, 'players', 'Kun\s.+', 'Sergio Agüero')
ingleses=subChar(ingleses, 'players', 'Su.+', 'Suárez')
ingleses=subChar(ingleses, 'players', 'Ma.+', 'Mané')
#ingleses


In [65]:
pichichi=subChar(campeones, 'players', '\s\[[0-9]+\]', '')
pichichi



,season,players,goals
0,2000,Salva Ballesta,27
1,2001,Raúl González,24
2,2002,Diego Tristán,21
3,2003,Roy Makaay,29
4,2004,Ronaldo,24
5,2005,Diego Forlán​,25
6,2006,Samuel Eto'o,26
7,2007,Ruud van Nistelrooy,25
8,2008,Dani Güiza,27
9,2009,Diego Forlán,32


In [67]:
import sys
import argparse
import subprocess

def recibeConfig():
    parser = argparse.ArgumentParser(description='¿Será el pichichi el fichaje del verano?')
    parser.add_argument('--summer',
                        help='Summer transfer market between 2000-2019',
                        '--league',
                        help='Select league Bundesliga, LaLiga, Premier League, League One, Serie A, Eredivisie
                        )
                        
    args = parser.parse_args()
    print(args)
    return args

SyntaxError: EOL while scanning string literal (<ipython-input-67-e5df5de6b06f>, line 10)

In [68]:
#pichichi(season,players,goals), ingleses, franceses, alemanes, holandeses, italianos

In [69]:
df.columns

Index(['Name', 'Team_from', 'League_from', 'Team_to', 'Summer', 'Transfer_fee',
       'Name-season'],
      dtype='object')

In [70]:
df = changeType(df, 'Name', str)
pichichi = changeType(pichichi, 'players', str)
pichichi = changeType(pichichi, 'season', int)




In [71]:
'''
for row in df.iterrows():
    for players in pichichi['players']:
        for seas in pichichi['season']:
            if players == str(row) and str(seas) == str(row):
                print(row)
 '''   


"\nfor row in df.iterrows():\n    for players in pichichi['players']:\n        for seas in pichichi['season']:\n            if players == str(row) and str(seas) == str(row):\n                print(row)\n "

In [72]:
def combCol(DF, nameNew, column1, column2):
    DF[nameNew] = DF[column1]+DF[column2]
    return DF


In [73]:
'''
for names in df['Name']:
    for players in pichichi['players']:
        for summer in df['Summer']:
            for pichi in pichichi['season']:
                if names == players and summer == pichi:
                    print(True)
 '''   
#if (yourValue in df[cols]:
#        print('Found in '+cols)
#if df['Name'] in pichichi['players'] and df['Summer'] == pichichi['season']:
#    print(True)

"\nfor names in df['Name']:\n    for players in pichichi['players']:\n        for summer in df['Summer']:\n            for pichi in pichichi['season']:\n                if names == players and summer == pichi:\n                    print(True)\n "

In [74]:
todos = pd.concat([pichichi, ingleses, franceses, alemanes, holandeses, italianos])
todos

,season,players,goals
0,2000,Salva Ballesta,27
1,2001,Raúl González,24
2,2002,Diego Tristán,21
3,2003,Roy Makaay,29
4,2004,Ronaldo,24
...,...,...,...
14,2014,Ciro Immobile,22
15,2015,Mauro Icardi,22
17,2016,Gonzalo Higuain,36
18,2017,Edin Dzeko,29


In [75]:
bundT = changeType(alemanes, 'season', str)
frenchT = changeType(franceses, 'season', str)
hollT = changeType(holandeses, 'season', str)
premierT = changeType(ingleses, 'season', str)
spainT = changeType(pichichi, 'season', str)
italyT = changeType(italianos, 'season', str)

In [76]:
bundT = combCol(bundT, 'Comb', 'players', 'season')
frenchT = combCol(frenchT, 'Comb', 'players', 'season')
hollT = combCol(hollT, 'Comb', 'players', 'season')
premierT = combCol(premierT, 'Comb', 'players', 'season')
spainT = combCol(spainT, 'Comb', 'players', 'season')
italyT = combCol(italyT, 'Comb', 'players', 'season')

In [79]:
premierT

,season,players,goals,Comb
22,2000,Kevin Phillips,30,Kevin Phillips2000
21,2001,Jimmy Floyd Hasselbaink,23,Jimmy Floyd Hasselbaink2001
20,2002,Thierry Henry,24,Thierry Henry2002
19,2003,Ruud van Nistelrooy,25,Ruud van Nistelrooy2003
18,2004,Thierry Henry,30,Thierry Henry2004
17,2005,Thierry Henry,25,Thierry Henry2005
16,2006,Thierry Henry,27,Thierry Henry2006
15,2007,Didier Drogba,20,Didier Drogba2007
14,2008,Cristiano Ronaldo,31,Cristiano Ronaldo2008
13,2009,Nicolas Anelka,19,Nicolas Anelka2009


In [ ]:
for name in df['Name-season']:
    for player in pichichi['cosa']:
        if name == player: 
            print(df.loc[df['Name-season'] == name])